<a href="https://colab.research.google.com/github/olivia-sp/study-pjt/blob/main/gpu_ex_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install this package to use Colab's GPU for training
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.4.1.50-1+cuda11.6).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [2]:
!pip install -U tensorboard_plugin_profile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [4]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from datetime import datetime

# Load the data

# Get current working directory
current_dir = os.getcwd() 
current_dir


data_train, data_test = tf.keras.datasets.fashion_mnist.load_data()


# Parse images and labels
(images_train, labels_train) = data_train
(images_test, labels_test) = data_test



# Remember to inherit from the correct class
class myCallback(tf.keras.callbacks.Callback):
        # Define the correct function signature for on_epoch_end
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('accuracy') is not None and logs.get('accuracy') > 0.97:
                print("\nReached 99% accuracy so cancelling training!") 
                
                # Stop training once the above condition is met
                self.model.stop_training = True


In [7]:
def reshape_and_normalize(images):
    
    ### START CODE HERE

    # Reshape the images to add an extra dimension
    images = images.reshape(60000, 28, 28, 1)
    
    # Normalize pixel values
    images = images / 255.0
    
    ### END CODE HERE

    return images


def solution_model():
    fashion_mnist = tf.keras.datasets.fashion_mnist
    (training_images, training_labels), _ = fashion_mnist.load_data()


    (training_images, _), _ = fashion_mnist.load_data()
    training_images = reshape_and_normalize(training_images)

    print(f"Maximum pixel value after normalization: {np.max(training_images)}\n")
    print(f"Shape of training set after reshaping: {training_images.shape}\n")
    print(f"Shape of one image after reshaping: {training_images[0].shape}")



    # Instantiate the callback class
    callbacks = myCallback()
    #
    model = tf.keras.models.Sequential([ 
        tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ]) 

      
    # Compile the model
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy',  
                  metrics=['accuracy']) 
  
    # Create a TensorBoard callback
    logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')

    # Fit the model for 10 epochs adding the callbacks
    # and save the training history
    model.fit(training_images, training_labels, epochs=3, callbacks=[tboard_callback])



    # YOUR CODE HERE
    return model


In [8]:
tf.config.experimental.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [9]:
# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.

if __name__ == '__main__':
  # gpus = tf.config.list_physical_devices('GPU')
  # if gpus:
  #   try:
  #     tf.config.experimental.set_memory_growth(gpus[0], True)
  #   except RuntimeError as e:
  #     # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
  #     print(e)
  
  os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private' # 추가
  model = solution_model()
  # Load the TensorBoard notebook extension.
  %reload_ext tensorboard

    #model.save("mymodel.h5")wa
  #with tf.device('/device:GPU:0'):
    

Maximum pixel value after normalization: 1.0

Shape of training set after reshaping: (60000, 28, 28, 1)

Shape of one image after reshaping: (28, 28, 1)
Epoch 1/3
1875/1875 [==============================] - 14s 6ms/step - loss: 0.4059 - accuracy: 0.8528
Epoch 2/3
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2737 - accuracy: 0.9003
Epoch 3/3
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2323 - accuracy: 0.9159


In [11]:
%reload_ext tensorboard